In [67]:
%%capture
!pip install streamlit

In [68]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import streamlit as st
import re
import math

## Abstract

`TODO`

## Introduction

`TODO`

## Preprocessing

Data can be found [here](http://www.astronexus.com/hyg).

In [69]:
# Read data with pandas
stars = pd.read_csv("https://raw.githubusercontent.com/astronexus/HYG-Database/main/hyg/v3/hyg_v37.csv")

In [70]:
# Select subset of columns
columns = ["id", "proper", "dist", "absmag", "spect", "ci", "x", "y", "z"]
stars = stars[columns]

# Select subset of rows (distance is in parsecs)
stars = stars[stars["dist"] < 1e3]
stars = stars.dropna(0, subset = ["ci", "absmag"])

# Convert spectrum to string
stars["spect"] = stars["spect"].apply(lambda x: str(x))

<ipython-input-70-e33ef56ccc5b>:7: FutureWarning:

In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.



Spectrum. See this [link](https://it.wikipedia.org/wiki/Classificazione_stellare).

In [71]:
# Example of spectra
np.unique(stars["spect"])[1:20]

array(['A', 'A comp SB', 'A+...', 'A-F', 'A...', 'A0', 'A0+...', 'A0...',
       'A0/1V + K1/2', 'A0/2V + A9:', 'A0/A1', 'A0/A1III', 'A0/A1IV',
       'A0/A1IV/V', 'A0/A1IVp...', 'A0/A1Ia', 'A0/A1V', 'A0/A1V:',
       'A0/A1Vn'], dtype=object)

[Link](https://regex101.com/) for regex.

In [72]:
# Infer star subtype
def find_subtype(x):
  r = re.findall("^[a-z]\d", str.lower(x))
  result = r[0][-1] if r else "nan"
  return str(result)

stars["subtype"] = stars["spect"].apply(lambda x: find_subtype(x))
# Drop missing
stars = stars[stars["subtype"] != "nan"]
assert(len(np.unique(stars["subtype"]) == 10))

In [73]:
stars.head()

,id,proper,dist,absmag,spect,ci,x,y,z,subtype
0,0,Sol,0.0000,4.850,G2V,0.656,0.000005,0.000000,0.000000,2
1,1,NaN,219.7802,2.390,F5,0.482,219.740502,0.003449,4.177065,5
2,2,NaN,47.9616,5.866,K3V,0.999,45.210918,0.003365,-16.008996,3
3,3,NaN,442.4779,-1.619,B9,-0.019,344.552785,0.030213,277.614965,9
4,4,NaN,134.2282,2.421,F0V,0.370,82.835513,0.012476,-105.619540,0


In [74]:
# Describe dataset
stars.describe()

,id,dist,absmag,ci,x,y,z
count,102955.000000,102955.000000,102955.000000,102955.000000,102955.000000,102955.000000,102955.000000
mean,59134.077548,249.272941,1.896419,0.693372,-0.092112,1.792605,-7.472068
std,34290.046141,188.794196,2.083714,0.470762,165.275924,180.544818,194.445965
min,0.000000,0.000000,-7.223000,-0.400000,-975.513096,-962.045493,-978.837073
25%,29168.500000,112.739600,0.501500,0.356000,-79.023717,-81.427810,-96.538482
50%,59238.000000,198.019800,1.654000,0.600000,-0.134967,-1.399438,-3.919803
75%,88853.500000,330.033000,3.164000,1.057000,79.345041,82.432533,84.881229
max,119618.000000,990.099000,17.986000,5.460000,977.365049,966.239750,970.242767


In [75]:
# Visualize points
## Visualize it -- Point cloud on Plotly
points = np.asarray(stars[["x", "y", "z"]])
colors = (points - np.min(points, axis=0))/(np.max(points, axis=0) - np.min(points, axis=0))
# colors[:,[0,1]] = 0

# Define a figure
fig = go.Figure(
  data=[
      # Define the data in the figure -- We have a 3D point Cloud
    go.Scatter3d(
      x=points[:,0], y=points[:,1], z=points[:,2],
      mode='markers',
      marker=dict(size=1, color=colors)
)
],
layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    xaxis=go.layout.XAxis(linecolor="black", linewidth=1, mirror=True),
    yaxis=go.layout.YAxis(linecolor="black", linewidth=1, mirror=True),
    margin=go.layout.Margin(l=50, r=50, b=100, t=100, pad=4),
)

)
# fig.show()

## Graph Generation

## Spectral Clustering

## Graph Neural Network (GNN)

## Evaluation

## Conclusion